In [34]:
import ROOT as r
from ROOT import TH1D, TFile, TCanvas, kSpring, kMagenta, kWhite, kBlack, TLegend, TLatex, THStack

import os

# Can use a macro to set a lot of style settings you want, experiments might have these to ensure nice uniformity. e.g. here we use the ATLAS one, but you could have your own to get a reliable setup you're happy with.
r.gROOT.LoadMacro("utils/AtlasStyle.C")
r.SetAtlasStyle()
r.gROOT.SetBatch(1)
r.gStyle.SetOptStat(0)
r.gStyle.SetPalette(112)
r.gStyle.SetTitleYOffset(1.1)
r.gStyle.SetErrorX(0)


luminosity_ifb = 10

def createDirs(path):
    """
    Function to create any directories needed to store the output of a function.

    Args:
        path (str): the path of directories you want to exist

    Returns:
        void
    """
    base = path.split("/")[0]
    dirs = path.split("/")[:]
    tmp_dir = ""
    for folder in dirs:
        tmp_dir = tmp_dir + "/" + folder
        if not os.path.isdir(base+"/"+tmp_dir):
            try:
                os.mkdir(base+"/"+tmp_dir)
                print("creating: ", base+"/"+tmp_dir)
            except OSError as error:
                print(error)

# Define out input and output paths, make sure the output path exists.
hist_path = "./histograms/GamGam_root/"
plot_path = "./plots/GamGam_root/"
createDirs(plot_path)


Applying ATLAS style settings...



In [ ]:
def plotStack(hists_in, xrange, plotdir, variable):
    """
    plot a stack of filled histograms

    Args:
        hists_in (dict(str,TH1)): str labels of input hists and the hists themselves.
        xrange (list(double)): min and max x-range.
        rebin (int): if you want to make the bins rebin* wider.
    """
    if len(xrange)<2: raise IndexError

    # define some config
    legenddict = {"ggfHiggs": "ggf H#gamma#gamma", 
                  "VBFHiggs": "VBF H#gamma#gamma"}
    colourdict = {"ggfHiggs": kSpring-5,
                  "VBFHiggs": kMagenta-6}

    # initialise our canvas
    canvas = TCanvas(variable, variable, 200, 10, 700, 750)
    canvas.cd(1) # point current root tdirectory and Draw()s to it
    canvas.SetTopMargin(0.03)
    canvas.SetRightMargin(0.05)
    canvas.SetLeftMargin(0.12)
    canvas.SetFillColor(kWhite)
    canvas.SetFillStyle(0)
    canvas.SetLineWidth(1)
    canvas.SetLogy(1)

    # intiialise our legend
    legend = TLegend(0.7, 0.8, 0.95, 0.94)
    legend.SetTextFont(42)
    legend.SetTextSize(0.036)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
  
    # initalise our Stack
    stack = THStack("signal_stack", ";;Events / bin")

    # read in and config our histograms
    hist = {}
    for label, hist_in in hists_in.items():
        hist[label] = hist_in
        hist[label].Sumw2()
        hist[label].GetXaxis().SetRangeUser(xrange[0], xrange[1])
        hist[label].SetFillColor(colourdict[label])
        stack.Add(hist[label])
        legend.AddEntry(hist[label], legenddict[label], "f")

    # Get the stack total
    stack_total = stack.GetStack().Last().Clone("total")
    stack_total.GetXaxis().SetRangeUser(xrange[0], xrange[1])
    stack_total.SetFillStyle(0) # 0 = hollow
     # Extract error on SM total
    total_error = stack_total.Clone("total_error")
    total_error.Sumw2()
    total_error.GetXaxis().SetRangeUser(xrange[0], xrange[1])
    total_error.SetFillStyle(3005)
    total_error.SetMarkerSize(0)
    total_error.SetFillColor(r.kBlack)
    total_error.SetLineColor(r.kBlack)
    legend.AddEntry(total_error,"Total","flp")

    # find the y range and set nice axis limits around it
    stack_total.GetYaxis().SetRangeUser(max(stack_total.GetMinimum(),0.001)*0.2,stack_total.GetMaximum()*10)
    
    # actually draw the histograms in the canvas
    stack_total.Draw("hist") # plot this first to set the y range
    stack.Draw("histf same")
    total_error.Draw("E3 same")

    # draw the legend
    legend.Draw("same")

    # add some text
    l=TLatex()
    l.SetNDC()
    l.SetTextFont(42)
    l.SetTextColor(kBlack)
    l.SetTextSize(0.036)
    lumi_string = '10 fb^{#minus1} #sqrt{s}=13 TeV'
    l.DrawLatex(0.2, 0.9, lumi_string)
 
    canvas.RedrawAxis()
    exts = [".pdf", ".png"]
    for ext in exts: canvas.SaveAs(plotdir + "Stack_" + variable + ext)


In [67]:
# config our samples and variables
samples_to_stack = ["VBFHiggs", "ggfHiggs"]
vars_to_plot = ["diphoton_mass"]

var_config = {
    "diphoton_mass":{
        "xrange": [0., 500.],
        "rebin": 2
    }
}

# retrieve the histogram files
files = {}
for sample in samples_to_stack:
    files[sample] = TFile(hist_path + sample + ".root", "READ")
    print(files[sample])
for var in vars_to_plot:
    # produce dict of our histograms per sample 
    hists_in = {}
    for sample in samples_to_stack:
        hists_in[sample] = files[sample].Get(var)
        print(hists_in[sample])
    # produce the stack plots
    plotStack(hists_in, var_config[var]["xrange"], plot_path, var, rebin=var_config[var]["rebin"])

Name: ./histograms/GamGam_root/VBFHiggs.root Title: 
Name: ./histograms/GamGam_root/ggfHiggs.root Title: 
Name: diphoton_mass Title: diphoton_mass NbinsX: 100
Name: diphoton_mass Title: diphoton_mass NbinsX: 100


Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: pdf file ./plots/GamGam_root/Stack_diphoton_mass.pdf has been created
Info in <TCanvas::Print>: png file ./plots/GamGam_root/Stack_diphoton_mass.png has been created


In [37]:
def plotWithFit(datahist, signalhist):
    c = TCanvas()
    data = datahist